## Import libraries

In [1]:
import folium
import pandas as pd
import json
from numpy import median
from folium import Choropleth, Circle, Marker, Icon, Map
import ast

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data/df_api.csv')

In [3]:
df.head()

,name,total_money_raised,offices_country_code,offices_state_code,offices_latitude,offices_longitude,offices_address_1,offices_address_2,offices_zip_code,vegan_rest,num_vegan_rest,preschool,num_preschool,starbucks,num_starbucks,airport,num_airport,clubs,num_clubs,weighted_punct
0,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",10,"[{'fsq_id': '4d370251e4b4a09380322936', 'categ...",3,"[{'fsq_id': '506c7556e4b03bdbe5afda50', 'categ...",8,"[{'fsq_id': '4fd2219fe4b03e7cee27e882', 'categ...",10,"[{'fsq_id': '60da36b2d61cae1ce8d45a04', 'categ...",10,824.98
1,Aviary,$19M,USA,NY,40.749376,-73.996435,243 West 30th Street,11th Floor,10001,"[{'fsq_id': '5f161e6f978e663e91ab7155', 'categ...",10,"[{'fsq_id': '4d370251e4b4a09380322936', 'categ...",2,"[{'fsq_id': '55a555fd498e1727ae82a345', 'categ...",10,"[{'fsq_id': '4adb6119f964a520a72621e3', 'categ...",10,"[{'fsq_id': '4cc0b629f82ebfb7b4af6f91', 'categ...",8,821.40
2,Gilt Groupe,$236M,USA,NY,40.747270,-73.980064,2 Park Ave Fl 4,NaN,10016-5602,"[{'fsq_id': '4b02f734f964a520814b22e3', 'categ...",10,"[{'fsq_id': '52712a6911d25f0e0dfa94cb', 'categ...",1,"[{'fsq_id': '4aa01922f964a520483e20e3', 'categ...",10,"[{'fsq_id': '4a5623d5f964a52081b41fe3', 'categ...",10,"[{'fsq_id': '741d73901e18012f487f54d8', 'categ...",4,732.10
3,99designs,$35M,USA,CA,37.795531,-122.400598,447 Battery St.,3rd Floor,94111,"[{'fsq_id': '4b391baef964a520605625e3', 'categ...",10,"[{'fsq_id': '558a380a498e13330910aef6', 'categ...",1,"[{'fsq_id': '6167c8968b400716f1d65e0b', 'categ...",7,"[{'fsq_id': '535ece06498e564c67f2f049', 'categ...",9,"[{'fsq_id': '4f32bd4d19836c91c7f53884', 'categ...",7,699.97
4,Minted,$52.7M,USA,CA,37.797435,-122.403175,747 Front Street,2nd Floor,94111,"[{'fsq_id': '4b391baef964a520605625e3', 'categ...",10,"[{'fsq_id': '5cc0a9cac0c0840025deceea', 'categ...",3,"[{'fsq_id': '49d0da91f964a520395b1fe3', 'categ...",3,"[{'fsq_id': 'eb55470d48024203bb09894b', 'categ...",3,"[{'fsq_id': '55ed1abb498e9e55d2e0edfc', 'categ...",9,607.15


In [4]:
df_subset = df[df['weighted_punct'] > 600]

In [5]:
df_subset['vegan_rest'] = df_subset['vegan_rest'].apply(ast.literal_eval)
df_subset['preschool'] = df_subset['preschool'].apply(ast.literal_eval)
df_subset['starbucks'] = df_subset['starbucks'].apply(ast.literal_eval)
df_subset['airport'] = df_subset['airport'].apply(ast.literal_eval)
df_subset['clubs'] = df_subset['clubs'].apply(ast.literal_eval)

In [6]:
def pre_explode(row, name):
    new_list = []
    for i in row[name]:
        lat = i["geocodes"]["main"]["latitude"]
        lon = i["geocodes"]["main"]["longitude"]
        new_list.append((lat, lon))
    return new_list

In [28]:
# Vegan restaurant
df_subset_vegan_rest = df_subset[['name', 'offices_state_code', 'offices_latitude', 'offices_longitude', 'vegan_rest', 'num_vegan_rest']]
df_subset_vegan_rest['vegan_rest_coord'] = df_subset_vegan_rest.apply(lambda row: pre_explode(row, 'vegan_rest'), axis=1)
df_subset_vegan_rest = df_subset_vegan_rest.explode('vegan_rest_coord')
df_subset_vegan_rest = df_subset_vegan_rest.reset_index(drop=True)
df_subset_vegan_rest[['vegan_rest_lat', 'vegan_rest_lon']] = df_subset_vegan_rest['vegan_rest_coord'].apply(lambda x: pd.Series([x[0], x[1]]))
df_subset_vegan_rest = df_subset_vegan_rest.drop_duplicates(subset='vegan_rest_coord', keep='first')

In [9]:
# Preschool
df_subset_preschool = df_subset[['name', 'offices_state_code', 'offices_latitude', 'offices_longitude', 'preschool', 'num_preschool']]
df_subset_preschool['preschool_coord'] = df_subset_preschool.apply(lambda row: pre_explode(row, 'preschool'), axis=1)
df_subset_preschool = df_subset_preschool.explode('preschool_coord')
df_subset_preschool = df_subset_preschool.reset_index(drop=True)
df_subset_preschool[['preschool_lat', 'preschool_lon']] = df_subset_preschool['preschool_coord'].apply(lambda x: pd.Series([x[0], x[1]]))
df_subset_preschool = df_subset_preschool.drop_duplicates(subset='preschool_coord', keep='first')

In [11]:
# Starbucks
df_subset_starbucks = df_subset[['name', 'offices_state_code', 'offices_latitude', 'offices_longitude', 'starbucks', 'num_starbucks']]
df_subset_starbucks['starbucks_coord'] = df_subset_starbucks.apply(lambda row: pre_explode(row, 'starbucks'), axis=1)
df_subset_starbucks = df_subset_starbucks.explode('starbucks_coord')
df_subset_starbucks = df_subset_starbucks.reset_index(drop=True)
df_subset_starbucks[['starbucks_lat', 'starbucks_lon']] = df_subset_starbucks['starbucks_coord'].apply(lambda x: pd.Series([x[0], x[1]]))
df_subset_starbucks = df_subset_starbucks.drop_duplicates(subset='starbucks_coord', keep='first')

In [12]:
# Airport
df_subset_airport = df_subset[['name', 'offices_state_code', 'offices_latitude', 'offices_longitude', 'airport', 'num_airport']]
df_subset_airport['airport_coord'] = df_subset_airport.apply(lambda row: pre_explode(row, 'airport'), axis=1)
df_subset_airport = df_subset_airport.explode('airport_coord')
df_subset_airport = df_subset_airport.reset_index(drop=True)
df_subset_airport[['airport_lat', 'airport_lon']] = df_subset_airport['airport_coord'].apply(lambda x: pd.Series([x[0], x[1]]))
df_subset_airport = df_subset_airport.drop_duplicates(subset='airport_coord', keep='first')

In [13]:
# Clubs
df_subset_clubs = df_subset[['name', 'offices_state_code', 'offices_latitude', 'offices_longitude', 'clubs', 'num_clubs']]
df_subset_clubs['clubs_coord'] = df_subset_clubs.apply(lambda row: pre_explode(row, 'clubs'), axis=1)
df_subset_clubs = df_subset_clubs.explode('clubs_coord')
df_subset_clubs = df_subset_clubs.reset_index(drop=True)
df_subset_clubs[['clubs_lat', 'clubs_lon']] = df_subset_clubs['clubs_coord'].apply(lambda x: pd.Series([x[0], x[1]]))
df_subset_clubs = df_subset_clubs.drop_duplicates(subset='clubs_coord', keep='first')

In [42]:
df_subset_vegan_rest['category_name'] = 'Vegan'
df_subset_preschool['category_name'] = 'Prechools'
df_subset_starbucks['category_name'] = 'Starbucks'
df_subset_airport['category_name'] = 'Airports'
df_subset_clubs['category_name'] = 'Clubs'

In [46]:
near_office = pd.concat([df_subset_vegan_rest, df_subset_preschool, df_subset_starbucks, df_subset_airport, df_subset_clubs], ignore_index=True)

In [47]:
near_office

,name,offices_state_code,offices_latitude,offices_longitude,vegan_rest,num_vegan_rest,vegan_rest_coord,vegan_rest_lat,vegan_rest_lon,group,...,airport,num_airport,airport_coord,airport_lat,airport_lon,clubs,num_clubs,clubs_coord,clubs_lat,clubs_lon
0,Netbiscuits,NY,40.746754,-73.995323,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",10.0,"(40.744946, -73.990249)",40.744946,-73.990249,Vegan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Netbiscuits,NY,40.746754,-73.995323,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",10.0,"(40.743323, -73.994378)",40.743323,-73.994378,Vegan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Netbiscuits,NY,40.746754,-73.995323,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",10.0,"(40.745153, -73.994538)",40.745153,-73.994538,Vegan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Netbiscuits,NY,40.746754,-73.995323,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",10.0,"(40.746151, -73.997547)",40.746151,-73.997547,Vegan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Netbiscuits,NY,40.746754,-73.995323,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",10.0,"(40.745543, -73.994073)",40.745543,-73.994073,Vegan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,99designs,CA,37.795531,-122.400598,NaN,NaN,NaN,NaN,NaN,Clubs,...,NaN,NaN,NaN,NaN,NaN,"[{'fsq_id': '4f32bd4d19836c91c7f53884', 'categ...",7.0,"(37.797885, -122.405014)",37.797885,-122.405014
129,99designs,CA,37.795531,-122.400598,NaN,NaN,NaN,NaN,NaN,Clubs,...,NaN,NaN,NaN,NaN,NaN,"[{'fsq_id': '4f32bd4d19836c91c7f53884', 'categ...",7.0,"(37.796492, -122.403999)",37.796492,-122.403999
130,99designs,CA,37.795531,-122.400598,NaN,NaN,NaN,NaN,NaN,Clubs,...,NaN,NaN,NaN,NaN,NaN,"[{'fsq_id': '4f32bd4d19836c91c7f53884', 'categ...",7.0,"(37.798086, -122.403663)",37.798086,-122.403663
131,Minted,CA,37.797435,-122.403175,NaN,NaN,NaN,NaN,NaN,Clubs,...,NaN,NaN,NaN,NaN,NaN,"[{'fsq_id': '55ed1abb498e9e55d2e0edfc', 'categ...",9.0,"(37.798253, -122.405322)",37.798253,-122.405322


In [44]:
office_nearby = Map(location = [51, 0], zoom_start = 11.4)
vegan_group = folium.FeatureGroup(name = f"Vegan restaurants ({near_office[near_office['category_name'] == 'Vegan'].shape[0]})")
starbucks_group = folium.FeatureGroup(name = f"Starbucks ({near_office[near_office['category_name'] == 'Starbucks'].shape[0]})")
schools_group = folium.FeatureGroup(name = f"Schools ({near_office[near_office['category_name'] == 'Schools'].shape[0]})")
airports_group = folium.FeatureGroup(name = f"Airports ({near_office[near_office['category_name'] == 'Airports'].shape[0]})")
clubs_group = folium.FeatureGroup(name = f"Clubs ({near_office[near_office['category_name'] == 'Clubs'].shape[0]})")

In [45]:
for index, row in near_office.iterrows():

    city = {
        "location": [row["latitude"], row["longitude"]],
        "tooltip": row["name"]
    }


    if row["category_name"] == "Vegan":
        icon = Icon (
            color = "lightgreen",
            prefix="fa",
            icon="leaf"

        )
    elif row["category_name"] == "Starbucks":
        icon = Icon (
            color = "darkgreen",
            prefix="fa",
            icon="coffee"
        )
    elif row["category_name"] == "Schools":
        icon = Icon (
            color = "blue",
            prefix="fa",
            icon="school"
        )
    elif row["category_name"] == "Clubs":
        icon = Icon (
            color = "darkblue",
            prefix="fa",
            icon="martini-glass"
        )
    else:
        icon = Icon(
            color = "grey",
            prefix="fa",
            icon="plane"
        )

    if row["category_name"] == "Vegan":
        new_marker.add_to(vegan_group)
    elif row["category_name"] == "Starbucks":
        new_marker.add_to(starbucks_group)
    elif row["category_name"] == "Schools":
        new_marker.add_to(preschools_group)
    elif row["category_name"] == "Clubs":
        new_marker.add_to(clubs_group)
    else:
        new_marker.add_to(airports_group)

pet_group.add_to(office_nearby)
vegan_group.add_to(office_nearby)
starbucks_group.add_to(office_nearby)
preschools_group.add_to(office_nearby)
airports_group.add_to(office_nearby)
clubs_group.add_to(office_nearby)


add_marker(office_nearby)

folium.LayerControl(collapsed=False, position="topleft").add_to(office_nearby)
office_nearby


KeyError: 'latitude'

In [14]:
p1_lat_NY, p1_lon_NY, p2_lat_NY, p2_lon_NY, p3_lat_NY, p3_lon_NY = df['offices_latitude'][0], df['offices_longitude'][0], df['offices_latitude'][1], df['offices_longitude'][1], df['offices_latitude'][2], df['offices_longitude'][2]

In [15]:
middle_lat_NY = median([p1_lat_NY, p2_lat_NY, p3_lat_NY])
middle_lon_NY = median([p1_lon_NY, p2_lon_NY, p3_lon_NY])

In [16]:
testmap_NY = folium.Map(location=[middle_lat_NY, middle_lon_NY], zoom_start=13)

In [17]:
folium.Circle(location=[p1_lat_NY, p1_lon_NY], popup='Point 1A', fill_color='#ff0000', radius=500, weight=2, color="#000000").add_to(testmap_NY)
folium.Circle(location=[p2_lat_NY, p2_lon_NY], popup='Point 1B', fill_color='#00ff00', radius=500, weight=2, color="#000000").add_to(testmap_NY)
folium.Circle(location=[p3_lat_NY, p3_lon_NY], popup='Point 1C', fill_color='#0000ff', radius=500, weight=2, color="#000000").add_to(testmap_NY)

In [18]:
for index, row in df_subset_airport.iterrows():

    airport = {"location": [row["airport_lat"], row["airport_lon"]]}

    if row['offices_state_code'] == 'NY':
        icon_airport = Icon (
                icon = "plane",
                prefix = "fa",
                color = "gray",
                icon_color = "white")
            
    airport_marker = Marker(**airport, icon = icon_airport)
    airport_marker.add_to(testmap_NY)

In [19]:
for index, row in df_subset_clubs.iterrows():

    clubs = {"location": [row["clubs_lat"], row["clubs_lon"]]}

    if row['offices_state_code'] == 'NY':
        icon_clubs = Icon (
                icon = "star",
                prefix = "fa",
                color = "darkpurple",
                icon_color = "white")
            
    clubs_marker = Marker(**clubs, icon = icon_clubs)
    clubs_marker.add_to(testmap_NY)

In [20]:
for index, row in df_subset_starbucks.iterrows():

    starbucks = {"location": [row["starbucks_lat"], row["starbucks_lon"]]}

    if row['offices_state_code'] == 'NY':
        icon_starbucks = Icon (
                icon = "coffee",
                prefix = "fa",
                color = "green",
                icon_color = "white")
            
    starbucks_marker = Marker(**starbucks, icon = icon_starbucks)
    starbucks_marker.add_to(testmap_NY)

In [21]:
for index, row in df_subset_preschool.iterrows():

    preschool = {"location": [row["preschool_lat"], row["preschool_lon"]]}
            
    if row['offices_state_code'] == 'NY':
        icon_preschool = Icon (
                icon = "child",
                prefix = "fa",
                color = "blue",
                icon_color = "white")  

    preschool_marker = Marker(**preschool, icon= icon_preschool)
    preschool_marker.add_to(testmap_NY)

In [22]:
for index, row in df_subset_vegan_rest.iterrows():

    vegan_rest = {"location": [row["vegan_rest_lat"], row["vegan_rest_lon"]]}

    if row['offices_state_code'] == 'NY':
        icon_vegan_rest = Icon (
                icon = "cutlery",
                prefix = "fa",
                color = "orange",
                icon_color = "white")
            
    vegan_rest_marker = Marker(**vegan_rest, icon = icon_vegan_rest)
    vegan_rest_marker.add_to(testmap_NY)

In [26]:
folium.LayerControl(collapsed=False, position="topleft").add_to(testmap_NY)

In [27]:
testmap_NY

In [24]:
"""for index, row in df.iterrows():
    
    all_together = {"location": [row["offices_latitude"], row["offices_longitude"]]}
            
    # 1. Check for the conditions of Festividad: to get the diferent Icon
    if row["num_vegan_rest"] != None and row['offices_state_code'] == 'NY':
        icon = Icon (
                icon = "laptop",
                prefix = "fa",
                color = "red",
                icon_color = "white")
        
    elif row["num_preschool"] != None and row['offices_state_code'] == 'NY':
        icon = Icon (
                icon = "moon-o",
                prefix = "fa",
                color = "white",
                icon_color = "black")
        
    elif row["num_starbucks"] != None and row['offices_state_code'] == 'NY':
        icon = Icon (
                icon = "cutlery",
                prefix = "fa",
                color = "green",
                icon_color = "white")
        
    elif row["num_airport"] != None and row['offices_state_code'] == 'NY':
        icon = Icon (
                icon = "cutlery",
                prefix = "fa",
                color = "green",
                icon_color = "white")
        
    elif row["num_clubs"] != None and row['offices_state_code'] == 'NY':
        icon = Icon (
                icon = "cutlery",
                prefix = "fa",
                color = "green",
                icon_color = "white")
    
    # 2. With the icon: I pass that to the Marker

    new_marker = Marker(**all_together, icon = icon)
    
    # 3. Add a Marker per row
    
    new_marker.add_to(testmap_NY)

testmap_NY"""

'for index, row in df.iterrows():\n    \n    all_together = {"location": [row["offices_latitude"], row["offices_longitude"]]}\n            \n    #\xa01. Check for the conditions of Festividad: to get the diferent Icon\n    if row["num_vegan_rest"] != None and row[\'offices_state_code\'] == \'NY\':\n        icon = Icon (\n                icon = "laptop",\n                prefix = "fa",\n                color = "red",\n                icon_color = "white")\n        \n    elif row["num_preschool"] != None and row[\'offices_state_code\'] == \'NY\':\n        icon = Icon (\n                icon = "moon-o",\n                prefix = "fa",\n                color = "white",\n                icon_color = "black")\n        \n    elif row["num_starbucks"] != None and row[\'offices_state_code\'] == \'NY\':\n        icon = Icon (\n                icon = "cutlery",\n                prefix = "fa",\n                color = "green",\n                icon_color = "white")\n        \n    elif row["num_air

In [25]:
"""for index, row in df.iterrows():
    
    all_together = {"location": [row["offices_latitude"], row["offices_longitude"]]}
            
    # 1. Check for the conditions of Festividad: to get the diferent Icon
    if row["num_vegan_rest"] != None:
        icon = Icon (
                icon = "laptop",
                prefix = "fa",
                color = "red",
                icon_color = "white")
      
    # 2. With the icon: I pass that to the Marker

    new_marker = Marker(**all_together, icon = icon)
    
    # 3. Add a Marker per row
    
    new_marker.add_to(testmap_NY)

testmap_NY"""

'for index, row in df.iterrows():\n    \n    all_together = {"location": [row["offices_latitude"], row["offices_longitude"]]}\n            \n    #\xa01. Check for the conditions of Festividad: to get the diferent Icon\n    if row["num_vegan_rest"] != None:\n        icon = Icon (\n                icon = "laptop",\n                prefix = "fa",\n                color = "red",\n                icon_color = "white")\n      \n    # 2. With the icon: I pass that to the Marker\n\n    new_marker = Marker(**all_together, icon = icon)\n    \n    # 3. Add a Marker per row\n    \n    new_marker.add_to(testmap_NY)\n\ntestmap_NY'